## Train a Gragh Convolutional Surrogate Model

In [3]:
import glob
import os
import sys
from time import time
import numpy as np
import pandas as pd
import torch
import torch_geometric as tg
import torch.nn.functional as F
import altair as alt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle

sys.path.append('/home/ewhalen/projects/caeSurrogateUtility')
import caeUtility as cu

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss

In [4]:
model = FeaStNet()